# Integrating YOLOv3 in DeepStream

# Set up the model

In [1]:
! ../model/prebuild.sh

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      
yolov3.weights      100%[===================>] 236.52M  12.4MB/s    in 21s     


In [2]:
!make -C ../source_code/nvdsinfer_custom_impl_Yolo

make: Entering directory '/opt/nvidia/deepstream/deepstream-5.0/yolov3/source_code/nvdsinfer_custom_impl_Yolo'
make: Nothing to be done for 'all'.
make: Leaving directory '/opt/nvidia/deepstream/deepstream-5.0/yolov3/source_code/nvdsinfer_custom_impl_Yolo'


# Building the pipeline 

In [3]:
# Import Required Libraries 
import sys
sys.path.append('../source_code')
import gi
import time
gi.require_version('Gst', '1.0')
from gi.repository import GObject, Gst, GLib
from common.bus_call import bus_call
import pyds

# Defining the Class Labels
PGIE_CLASS_ID_PERSON = 0
PGIE_CLASS_ID_BICYCLE = 1
PGIE_CLASS_ID_CAR = 2
PGIE_CLASS_ID_BUS = 5
PGIE_CLASS_ID_TRAIN = 6
PGIE_CLASS_ID_TRUCK = 7
PGIE_CLASS_ID_BACKPACK = 24
PGIE_CLASS_ID_SURFBOARD = 37


# Defining the input output video file 
INPUT_VIDEO_NAME  = '../videos/sample_720p.h264'
OUTPUT_VIDEO_NAME = "../videos/out.mp4"

In [4]:
## Make Element or Print Error and any other detail
def make_elm_or_print_err(factoryname, name, printedname, detail=""):
  print("Creating", printedname)
  elm = Gst.ElementFactory.make(factoryname, name)
  if not elm:
     sys.stderr.write("Unable to create " + printedname + " \n")
  if detail:
     sys.stderr.write(detail)
  return elm

#### Initialise GStreamer and Create an Empty Pipeline

In [5]:
# Standard GStreamer initialization
Gst.init(None)


# Create Gstreamer elements
# Create Pipeline element that will form a connection of other elements
print("Creating Pipeline \n ")
pipeline = Gst.Pipeline()

if not pipeline:
    sys.stderr.write(" Unable to create Pipeline \n")

Creating Pipeline 
 


#### Create Elements that are required for our pipeline 

In [6]:
# Creating elements required for the pipeline
# Source element for reading from file
source = make_elm_or_print_err("filesrc", "file-source","Source")
# Parse the data since the input is an elementary .h264 stream
h264parser = make_elm_or_print_err("h264parse", "h264-parser","h264 parse")
# For hardware accelerated decoding of the stream
decoder = make_elm_or_print_err("nvv4l2decoder", "nvv4l2-decoder","Nvv4l2 Decoder")
# Form batches from one or more sources
streammux = make_elm_or_print_err("nvstreammux", "Stream-muxer",'NvStreamMux')
# Run inference on the decoded stream, this property is set through a configuration file later
pgie = make_elm_or_print_err("nvinfer", "primary-inference" ,"pgie")
# Convert output stream to formatted buffer accepted by Nvosd
nvvidconv = make_elm_or_print_err("nvvideoconvert", "convertor","nvvidconv")
# Draw on the buffer
nvosd = make_elm_or_print_err("nvdsosd", "onscreendisplay","nvosd")
# Encode and save the OSD output
queue = make_elm_or_print_err("queue", "queue", "Queue")
# Convert output for saving
nvvidconv2 = make_elm_or_print_err("nvvideoconvert", "convertor2","nvvidconv2")
# Save as video file
encoder = make_elm_or_print_err("avenc_mpeg4", "encoder", "Encoder")
# Parse output from encoder
codeparser = make_elm_or_print_err("mpeg4videoparse", "mpeg4-parser", 'Code Parser')
# Create a container
container = make_elm_or_print_err("qtmux", "qtmux", "Container")
# Create sink for string the output
sink = make_elm_or_print_err("filesink", "filesink", "Sink")

Creating Source
Creating h264 parse
Creating Nvv4l2 Decoder
Creating NvStreamMux
Creating pgie
Creating nvvidconv
Creating nvosd
Creating Queue
Creating nvvidconv2
Creating Encoder
Creating Code Parser
Creating Container
Creating Sink


#### Set Properties in pipeline

In [7]:
# Set properties for elements
print("Playing file %s" %INPUT_VIDEO_NAME)
# Set input file
source.set_property('location', INPUT_VIDEO_NAME)
# Set input height, width, and batch size
streammux.set_property('width', 1920)
streammux.set_property('height', 1080)
streammux.set_property('batch-size', 1)
# Set timer (in microseconds) to wait after the first buffer is available
# to push the batch even if batch is never completely formed
streammux.set_property('batched-push-timeout', 4000000)
# Set configuration files for Nvinfer
pgie.set_property('config-file-path', "../configs/config_infer_primary_yoloV3.txt")
# Set encoder bitrate for output video
encoder.set_property("bitrate", 2000000)
# Set output file location, disable sync and async
sink.set_property("location", OUTPUT_VIDEO_NAME)
sink.set_property("sync", 0)
sink.set_property("async", 0)

Playing file ../videos/sample_720p.h264


#### Adding and Linking elements

In [8]:
# Add and link all elements to the pipeline
# Adding elements
print("Adding elements to Pipeline \n")

pipeline.add(source)
pipeline.add(h264parser)
pipeline.add(decoder)
pipeline.add(streammux)
pipeline.add(pgie)
pipeline.add(nvvidconv)
pipeline.add(nvosd)
pipeline.add(queue)
pipeline.add(nvvidconv2)
pipeline.add(encoder)
pipeline.add(codeparser)
pipeline.add(container)
pipeline.add(sink)

# Linking elements
# Order: source -> h264parser -> decoder -> streammux -> pgie ->
# -> vidconv -> osd -> queue -> vidconv2 -> encoder -> parser ->
# -> container -> sink

print("Linking elements in the Pipeline \n")
source.link(h264parser)
h264parser.link(decoder)


sinkpad = streammux.get_request_pad("sink_0")
if not sinkpad:
    sys.stderr.write(" Unable to get the sink pad of streammux \n")
# Create source pad from Decoder   
srcpad = decoder.get_static_pad("src")
if not srcpad:
    sys.stderr.write(" Unable to get source pad of decoder \n")
    
srcpad.link(sinkpad)
streammux.link(pgie)
pgie.link(nvvidconv)
nvvidconv.link(nvosd)
nvosd.link(queue)
queue.link(nvvidconv2)
nvvidconv2.link(encoder)
encoder.link(codeparser)
codeparser.link(container)
container.link(sink)

Adding elements to Pipeline 

Linking elements in the Pipeline 



True

In [9]:
# Create an event loop and feed GStreamer bus messages to it
loop = GLib.MainLoop()
bus = pipeline.get_bus()
bus.add_signal_watch()
bus.connect ("message", bus_call, loop)

1

## Working with the Metadata 

In [10]:
# Working with metadata
def osd_sink_pad_buffer_probe(pad,info,u_data):
    
    obj_counter = {
        PGIE_CLASS_ID_PERSON:0,
        PGIE_CLASS_ID_BICYCLE:0,
        PGIE_CLASS_ID_CAR:0,
        PGIE_CLASS_ID_BUS:0,
        PGIE_CLASS_ID_TRAIN:0,
        PGIE_CLASS_ID_TRUCK:0,
        PGIE_CLASS_ID_BACKPACK:0,
        PGIE_CLASS_ID_SURFBOARD:0
    }
    # Reset frame number and number of rectanges to zero
    frame_number=0
    num_rects=0
    
    gst_buffer = info.get_buffer()
    if not gst_buffer:
        print("Unable to get GstBuffer ")
        return

    # Retrieve metadata from gst_buffer
    # Note: since we use the pyds shared object library,
    # the input is the C address of gst_buffer
    batch_meta = pyds.gst_buffer_get_nvds_batch_meta(hash(gst_buffer))
    l_frame = batch_meta.frame_meta_list
    while l_frame is not None:
        try:
            frame_meta = pyds.NvDsFrameMeta.cast(l_frame.data)
        except StopIteration:
            break
        
        # Get frame number, number of rectangles to draw and object metadata
        frame_number=frame_meta.frame_num
        num_rects = frame_meta.num_obj_meta
        l_obj=frame_meta.obj_meta_list
        
        while l_obj is not None:
            try:
                obj_meta=pyds.NvDsObjectMeta.cast(l_obj.data)
            except StopIteration:
                break
            # Increment object class by 1 and set box border color to red
            obj_counter[obj_meta.class_id] += 1
            obj_meta.rect_params.border_color.set(0.0, 0.0, 1.0, 0.0)
            try: 
                l_obj=l_obj.next
            except StopIteration:
                break
        
        # Setting metadata display configuration
        # Acquire display meta object
        display_meta=pyds.nvds_acquire_display_meta_from_pool(batch_meta)
        display_meta.num_labels = 1
        py_nvosd_text_params = display_meta.text_params[0]
        # Set display text to be shown on screen
        # py_nvosd_text_params.display_text = "Frame Number={} Number of Objects={} Vehicle_count={} Person_count={}".format(frame_number, num_rects, obj_counter[PGIE_CLASS_ID_VEHICLE], obj_counter[PGIE_CLASS_ID_PERSON])
        # Set where the string will appear
        py_nvosd_text_params.x_offset = 10
        py_nvosd_text_params.y_offset = 12
        # Font, font colour and font size
        py_nvosd_text_params.font_params.font_name = "Serif"
        py_nvosd_text_params.font_params.font_size = 10
        # Set color (We used white)
        py_nvosd_text_params.font_params.font_color.set(1.0, 1.0, 1.0, 1.0)
        # Set text background colour (We used black)
        py_nvosd_text_params.set_bg_clr = 1
        py_nvosd_text_params.text_bg_clr.set(0.0, 0.0, 0.0, 1.0)
        # Print the display text in the console as well
        #print(pyds.get_string(py_nvosd_text_params.display_text))
        pyds.nvds_add_display_meta_to_frame(frame_meta, display_meta)
        
        
        try:
            l_frame=l_frame.next
        except StopIteration:
            break
    return Gst.PadProbeReturn.OK

In [11]:
# Adding probe to sinkpad of the OSD element
osdsinkpad = nvosd.get_static_pad("sink")
if not osdsinkpad:
    sys.stderr.write(" Unable to get sink pad of nvosd \n")
    
osdsinkpad.add_probe(Gst.PadProbeType.BUFFER, osd_sink_pad_buffer_probe, 0)

1

#### Play Pipeline

In [12]:
# Start the pipeline
print("Starting pipeline \n")
start_time = time.time()
pipeline.set_state(Gst.State.PLAYING)
try:
    loop.run()
except:
    pass
# Cleanup
pipeline.set_state(Gst.State.NULL)
print("--- %s seconds ---" % (time.time() - start_time))

Starting pipeline 

End-of-stream
--- 25.884523630142212 seconds ---


#### Convert and Display

In [13]:
# Convert video profile to be compatible with the Notebook
!ffmpeg -loglevel panic -y -an -i ../videos/out.mp4 -vcodec libx264 -pix_fmt yuv420p -profile:v baseline -level 3 ../videos/output.mp4

In [14]:
# Display the Output
from IPython.display import HTML
HTML("""
 <video width="640" height="480" controls>
 <source src="../videos/output.mp4"
 </video>
""".format())